In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from modules.env import MazeEnvironment, MazeVisualization
from utils.skeletonize import skeletonize_env, draw_scatter_plot_skeleton
import seaborn as sns
from modules.base import SRMB
from modules.model import TAGTransfer
import matplotlib.gridspec as gridspec
import matplotlib
import math
import itertools
from utils.plot import smooth_mean_std
from itertools import combinations


%matplotlib inline
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['ps.fonttype'] = 42
rcParams['font.sans-serif'] = ['Arial']  # 예시: Arial로 설정
rcParams['font.family'] = 'sans-serif'

sns.set_style(
    'ticks',
    rc={
        'xtick.direction': 'in',  # x축 tick을 안쪽으로
        'ytick.direction': 'in',  # y축 tick을 안쪽으로
    }
)
rcParams["figure.dpi"] = 600
# Set the font to Arial
big_font = 20
mid_font = 15
small_font = 12

indicator_font = big_font
legend_font = mid_font
title_font = mid_font
axis_font = mid_font
tick_font = small_font



subplot_scale_factor = 3

indicator_x_loc = -0.1
indicator_y_loc = 1.15

node_size = 300
edge_width = 4

In [2]:
junction_color = "#416788"
deadend_color = "#646464"
edge_color = "#E1DAD2"

palette = ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600']

In [3]:
num_row, num_column = 2, 4
figsize_x = np.min([num_column * 5, 20])
figsize_y = figsize_x * num_row / num_column
fig = plt.figure(figsize=(figsize_x, figsize_y))
gs = gridspec.GridSpec(num_row, num_column, figure=fig, height_ratios=[1 for _ in range(num_row)], width_ratios=[1 for _ in range(num_column)])

# 개별 subplot 생성
ax_grid_inner = fig.add_subplot(gs[0, 0])
ax_grid_outer = fig.add_subplot(gs[0, 1])
ax_grid_empty = fig.add_subplot(gs[0, 2])
ax_model_2 = fig.add_subplot(gs[0, 3:4])
ax_ls_1 = fig.add_subplot(gs[1, 0])
ax_ls_2 = fig.add_subplot(gs[1, 1])
ax_cs_1 = fig.add_subplot(gs[1, 2])
ax_cs_2 = fig.add_subplot(gs[1, 3])


env3 = MazeVisualization(7, 7)
nonblocks = [0, 1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 13, 20, 27, 34, 41, 48, 43, 44, 45, 46, 47, 10, 17, 24, 31, 38]
blocks = np.setdiff1d(np.arange(49), nonblocks)
env3.update_map(blocks=blocks)
# env.scale_map(2)
env3.set_index("doubled")
srmb = SRMB(7, 7)
srmb_bias = SRMB(7, 7)
srmb.update_map_structure(blocks=env3.blocks)
srmb_bias.update_map_structure(blocks=env3.blocks, rd_prob=0.9)
successormap_unbiased = srmb.return_successormap()
successormap_biased = srmb_bias.return_successormap()
srmb_empty = SRMB(7, 7)
srmb_empty.update_map_structure(blocks=[])
successormap_empty = srmb_empty.return_successormap()

vertex_nodes_3, deadend_nodes_3, edge_nodes_3, simplified_G_3, image_3, skeleton_3, _, _, _, _, _, _ = (
    skeletonize_env(env3)
)

node_mapping = {}
for node in simplified_G_3.nodes:
    node_mapping[(node)] = (11, 6) if node == (10, 6) else (node[0], 6)

draw_scatter_plot_skeleton(
    successormap_unbiased[env3.nonblocks][:, :2],
    vertex_nodes_3,
    deadend_nodes_3,
    edge_nodes_3,
    env3.nonblocks,
    save=False,
    sizes_per_type=[node_size, node_size, node_size],
    ax=ax_grid_inner,
    fig=fig,
)

scatter_plot = ax_grid_inner.collections[0]  
scatter_plot.set_color('gray')  

draw_scatter_plot_skeleton(
    successormap_empty[:, :2],
    [np.arange(49)],
    [],
    [],
    np.arange(49),
    save=False,
    sizes_per_type=[node_size, node_size, node_size],
    ax=ax_grid_empty,
    fig=fig,
)

scatter_plot = ax_grid_empty.collections[0]  
scatter_plot.set_color('gray')  

draw_scatter_plot_skeleton(
    successormap_unbiased[env3.nonblocks][:, :2],
    vertex_nodes_3,
    deadend_nodes_3,
    edge_nodes_3,
    env3.nonblocks,
    save=False,
    sizes_per_type=[node_size, node_size-100, node_size-200],
    ax=ax_grid_outer,
    fig=fig,
)

num_row, num_column = 9, 9
# for scale_factor in range(1, 6):
scale_factor = 5

env = MazeEnvironment(num_row, num_column)
blocks = []
for r in range(1, 8):
    for c in range(1, 4):
        blocks.append(env.index_to_state(r, c))
    for c in range(5, 8):
        blocks.append(env.index_to_state(r, c))
env.insert_block(blocks)
env.scale_map(scale_factor)
blocks = env.blocks

tag = TAGTransfer(num_row * scale_factor, num_column * scale_factor)
srmb = SRMB(num_row * scale_factor, num_column * scale_factor)

srmb.update_map_structure(blocks)
tag.set_map(srmb.sr)
tag.update_grid()

num_models = 5
color_order = [0, 3, 1, 2, 4]

color_list = [plt.cm.tab10(color_order[i]) for i in range(num_models)]
metric_list = ['spearmanr', 'gw_dist']
model_list = ['TAG-WithBorder', 'TAG-WithoutBorder', 'SR-MB', 'SR-Dyna', 'SR-TD']
linestyle_list = ['solid' for _ in range(8)]
model_index_list = [0, 3, 1, 2, 4]
metric_index = 1

for model_index in model_index_list:
    mean_data = np.load('data/inner_loop/processed_data/{}_{}_mean.npy'.format(
        metric_index, model_index
    ))
    std_data = np.load('data/inner_loop/processed_data/{}_{}_std.npy'.format(
        metric_index, model_index
    ))
    mean_data, std_data = smooth_mean_std(mean_data, std_data, 100)
    ax_ls_1.plot(
        np.arange(2000) * 10,
        mean_data,
        linestyle=linestyle_list[model_index],
        label=model_list[model_index],
        color=palette[np.where(np.array(model_index_list) == model_index)[0][0]],
        alpha=0.9,
        lw=4,
    )
    ax_ls_1.fill_between(
        np.arange(2000) * 10,
        mean_data - std_data,
        mean_data + std_data,
        alpha=0.2,
        color=palette[np.where(np.array(model_index_list) == model_index)[0][0]],
    )
ax_ls_1.tick_params(axis='x', labelsize=tick_font)
ax_ls_1.tick_params(axis='y', labelsize=tick_font)
ax_ls_1.set_xlabel('Timesteps', fontsize=axis_font)
ax_ls_1.set_xticks([0, 10000, 20000])
ax_ls_1.set_xticklabels(['0', '10k', '20k'], fontsize=tick_font)
ax_ls_1.set_xlim(0, 20010)
ax_ls_1.set_yticks([0, 1, 2, 3])
ax_ls_1.set_yticklabels(['0', '1', '2', '3'], fontsize=tick_font)
ax_ls_1.set_ylim(0, 3)
ax_ls_1.set_ylabel('Grid code training loss\n(GW distance)', fontsize=axis_font)
ax_ls_1.legend(frameon=False, fontsize=legend_font)
ax_ls_1.set_xlim(0, 21000)
ax_ls_1.set_aspect(20000 / 3)
ax_ls_1.spines['top'].set_visible(False)
ax_ls_1.spines['right'].set_visible(False)


ax_ls_2.set_ylim((0, 2))
for model_index_index in range(num_models):
    model_index = model_index_list[model_index_index]
    x_offset = 0.8 / num_models * model_index_index + 0.4 / num_models - 0.4
    mean_list = []
    std_list = []
    for bias_index in range(5):
        mean_data = np.load('data/inner_loop/processed_data/{}_{}_{}_bias_mean.npy'.format(
            metric_index, model_index, bias_index
        ))
        std_data = np.load('data/inner_loop/processed_data/{}_{}_{}_bias_std.npy'.format(
            metric_index, model_index, bias_index
        ))
        mean_list.append(mean_data[-1])
        std_list.append(std_data[-1])
    ax_ls_2.bar(np.arange(len(mean_list)) + x_offset, mean_list,
            label=model_list[model_index], color=palette[np.where(np.array(model_index_list) == model_index)[0][0]], width=0.8/num_models, edgecolor="black", yerr=std_list)
ax_ls_2.set_xlabel('Exploration policy Entropy', fontsize=axis_font)
ax_ls_2.set_xticks(np.arange(5), ['H0\n(Unbiased)', 'H1', 'H2', 'H3', 'H4\n(Biased)'])
ax_ls_2.set_yticks([0, 1, 2])
ax_ls_2.set_yticklabels(['0', '1', '2'], fontsize=tick_font)
ax_ls_2.tick_params(axis='x', labelsize=tick_font, length=0)
ax_ls_2.tick_params(axis='y', labelsize=tick_font)
ax_ls_2.set_ylabel('Grid code training loss\n(GW distance)', fontsize=axis_font)
ax_ls_2.legend(fontsize=legend_font, frameon=False)
ax_ls_2.set_aspect(5 / 2)
ax_ls_2.spines['top'].set_visible(False)
ax_ls_2.spines['right'].set_visible(False)


feature_names = ['corner', 'border', 'place', 'grid']

# Generate all combinations of corner, border, place, and grid (excluding the block feature which is always included)
feature_combinations = []
for r in range(0, len(feature_names) + 1):
    feature_combinations.extend(combinations(feature_names, r))  # Generate 2^4 combinations of features

# Define a mapping of features to abbreviations
feature_abbreviations = {"corner": "c", "border": "b", "place": "p", "grid": "g"}

feature_abbreviations_mod = {"corner": "C", "border": "B", "place": "P", "grid": "G"}

# Initialize lists to store data
testacc_v = []
testacc_no_v = []
testacc_p = []
testacc_no_p = []
testacc_g = []
testacc_no_g = []
testacc_b = []
testacc_no_b = []

# Initialize dictionary to store highest test accuracy for each combination
highest_testacc_data = {}

# Load data and separate based on presence of 'c', 'p', 'g', and 'b'
for combo in feature_combinations:
    if len(combo):
        combo_abbr = ''.join([feature_abbreviations[feat] for feat in combo])
        combo_abbr_upper = ''.join([feature_abbreviations_mod[feat] for feat in combo]).upper()
    else:
        combo_abbr = 'n'
        combo_abbr_upper = 'N'
    
    testacc = np.load('data/outer_loop/logs/' + combo_abbr + '_stacked_testacc.npy')
    highest_testacc = np.max(testacc, axis=1)
    highest_testacc_data[combo_abbr_upper] = [np.mean(highest_testacc), np.std(highest_testacc), highest_testacc]
    
    if 'c' in combo_abbr:
        testacc_v.append(testacc)
    else:
        testacc_no_v.append(testacc)
    
    if 'p' in combo_abbr:
        testacc_p.append(testacc)
    else:
        testacc_no_p.append(testacc)
    
    if 'g' in combo_abbr:
        testacc_g.append(testacc)
    else:
        testacc_no_g.append(testacc)
    
    if 'b' in combo_abbr:
        testacc_b.append(testacc)
    else:
        testacc_no_b.append(testacc)

# Convert lists to numpy arrays for easier manipulation
testacc_v = np.array(testacc_v)
testacc_no_v = np.array(testacc_no_v)
testacc_p = np.array(testacc_p)
testacc_no_p = np.array(testacc_no_p)
testacc_g = np.array(testacc_g)
testacc_no_g = np.array(testacc_no_g)
testacc_b = np.array(testacc_b)
testacc_no_b = np.array(testacc_no_b)

# Extract highest values
highest_testacc_v = np.max(testacc_v, axis=2).flatten()
highest_testacc_no_v = np.max(testacc_no_v, axis=2).flatten()
highest_testacc_p = np.max(testacc_p, axis=2).flatten()
highest_testacc_no_p = np.max(testacc_no_p, axis=2).flatten()
highest_testacc_g = np.max(testacc_g, axis=2).flatten()
highest_testacc_no_g = np.max(testacc_no_g, axis=2).flatten()
highest_testacc_b = np.max(testacc_b, axis=2).flatten()
highest_testacc_no_b = np.max(testacc_no_b, axis=2).flatten()

# Calculate mean and std
mean_testacc_c = np.mean(highest_testacc_v)
std_testacc_c = np.std(highest_testacc_v)
sem_testacc_v = np.std(highest_testacc_v, ddof=1) / np.sqrt(len(highest_testacc_v))
mean_testacc_no_c = np.mean(highest_testacc_no_v)
std_testacc_no_c = np.std(highest_testacc_no_v)
sem_testacc_no_v = np.std(highest_testacc_no_v, ddof=1) / np.sqrt(len(highest_testacc_no_v))

mean_testacc_p = np.mean(highest_testacc_p)
sem_testacc_p = np.std(highest_testacc_p, ddof=1) / np.sqrt(len(highest_testacc_p))
std_testacc_p = np.std(highest_testacc_p)
mean_testacc_no_p = np.mean(highest_testacc_no_p)
sem_testacc_no_p = np.std(highest_testacc_no_p, ddof=1) / np.sqrt(len(highest_testacc_no_p))
std_testacc_no_p = np.std(highest_testacc_no_p)

mean_testacc_g = np.mean(highest_testacc_g)
sem_testacc_g = np.std(highest_testacc_g, ddof=1) / np.sqrt(len(highest_testacc_g))
std_testacc_g = np.std(highest_testacc_g)
mean_testacc_no_g = np.mean(highest_testacc_no_g)
sem_testacc_no_g = np.std(highest_testacc_no_g, ddof=1) / np.sqrt(len(highest_testacc_no_g))
std_testacc_no_g = np.std(highest_testacc_no_g)

mean_testacc_b = np.mean(highest_testacc_b)
sem_testacc_b = np.std(highest_testacc_b, ddof=1) / np.sqrt(len(highest_testacc_b))
std_testacc_b = np.std(highest_testacc_b)
mean_testacc_no_b = np.mean(highest_testacc_no_b)
sem_testacc_no_b = np.std(highest_testacc_no_b, ddof=1) / np.sqrt(len(highest_testacc_no_b))
std_testacc_no_b = np.std(highest_testacc_no_b)



neural_code_letters = ["C", "B", "P", "G"]


def coalition_key(coalition):
    """
    coalition: feature 인덱스들의 집합 (예, {0, 2})
    반환: highest_testacc_data에서 사용된 키 문자열.
           빈 집합이면 'N' 반환, 아니면 neural_code_letters를 순서대로 결합.
    """
    if not coalition:
        return "N"
    return "".join([neural_code_letters[i] for i in sorted(coalition)])


def v(coalition, seed_idx):
    """
    characteristic function v: 지정된 coalition과 seed index에 대해 Test Accuracy 값을 반환.
    highest_testacc_data[key][2]에는 각 seed별 test accuracy 값들이 저장되어 있다고 가정.
    """
    key = coalition_key(coalition)
    # 여기서는 각 조합의 평균(test accuracy)을 seed별로 반환
    return highest_testacc_data[key][2][seed_idx]


def compute_shapley_values_for_seed(num_players, seed_idx):
    shapley_values = [0.0] * num_players
    for i in range(num_players):
        # i번 feature를 제외한 나머지 인덱스 목록
        others = [j for j in range(num_players) if j != i]
        # 가능한 모든 부분 집합(S)들에 대해
        for r in range(len(others) + 1):
            for S in itertools.combinations(others, r):
                S = set(S)
                weight = (
                    math.factorial(len(S))
                    * math.factorial(num_players - len(S) - 1)
                    / math.factorial(num_players)
                )
                marginal_contribution = v(S.union({i}), seed_idx) - v(S, seed_idx)
                shapley_values[i] += weight * marginal_contribution
    return shapley_values

seed_count = len(highest_testacc_data["N"][2])
num_codes = 4 

shapley_all = []

for seed in range(seed_count):
    sv_seed = compute_shapley_values_for_seed(num_codes, seed)
    shapley_all.append(sv_seed)

shapley_all = np.array(shapley_all)  # shape: (seed_count, num_codes)

mean_sv = np.mean(shapley_all, axis=0)
std_sv = np.std(shapley_all, axis=0)

colors_list = ["#808080" if value >= 0 else "#C0C0C0" for value in mean_sv]

ax_cs_1.bar(
    neural_code_letters,
    mean_sv,
    color=colors_list,
    edgecolor="black",
    yerr=std_sv,
    capsize=0,
)

ax_cs_1.set_xlabel("Neural Code", fontsize=14)
ax_cs_1.set_ylabel("Shapley value", fontsize=14)
ax_cs_1.axhline(0, color="black", linewidth=0.8)

# Sort the combinations based on highest test accuracy in descending order
sorted_combos = sorted(highest_testacc_data.items(), key=lambda x: x[1][0], reverse=True)

# Extract sorted combinations and their highest test accuracy values
sorted_combos_abbr = [item[0] for item in sorted_combos]
sorted_highest_testacc_mean = [item[1][0] for item in sorted_combos]
sorted_highest_testacc_std = [item[1][1] for item in sorted_combos]



colors_list = ['#808080' if 'C' in abbr or 'B' in abbr else '#C0C0C0' for abbr in sorted_combos_abbr]
# Create the bar plot
bars = ax_cs_2.bar(
    sorted_combos_abbr,
    sorted_highest_testacc_mean,
    color=colors_list,
    edgecolor="black",
    capsize=0,
    yerr=sorted_highest_testacc_std,)
ax_cs_2.set_xlabel('Feature Combinations', fontsize=axis_font)
ax_cs_2.set_ylabel('Test Accuracy', fontsize=axis_font)
ax_cs_2.tick_params(axis='x', labelsize=tick_font, rotation=90)
ax_cs_2.tick_params(axis='y', labelsize=tick_font)
ax_cs_2.set_yticks([0.85, 0.90, 0.95, 1.0])
ax_cs_2.set_yticklabels(['0.85', '0.90', '0.95', '1.00'], fontsize=tick_font)
ax_cs_2.set_ylim([0.85, 1.])
ax_cs_2.set_aspect(16 / 0.15 / 1.5)
ax_cs_2.spines['top'].set_visible(False)
ax_cs_2.spines['right'].set_visible(False)



# 여백 조정 (필요에 따라)
fig.subplots_adjust(wspace=0.5, hspace=0.5)
plt.savefig('./figure/figure_2.svg', bbox_inches='tight', transparent=True)
plt.show()